In [2]:
import pandas as pd
import sqlite3

#incarcam datele
results = pd.read_csv("results.csv")
goalscorers = pd.read_csv("goalscorers.csv")
shootouts = pd.read_csv("shootouts.csv")

In [3]:
def connect_to_database(database_name):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    return conn, cursor

# Conectare la baza de date
conn, cursor = connect_to_database('football.db')

results.to_sql('results',conn, if_exists='replace', index=False)
goalscorers.to_sql('goalscorers',conn, if_exists='replace', index=False)
shootouts.to_sql('shootouts', conn, if_exists='replace', index=False)

#inchidem conexiunea la baza de date
conn.close()